## Module 6 Asignment: using Pandas and custom Decision Tree
# Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

* Implement binary decision trees with different early stopping methods.
* Compare models with different stopping parameters.
* Visualize the concept of overfitting in decision trees.

Import pandas.

In [50]:
import pandas as pd

# Load LendingClub Dataset

**1.** This assignment will use the [LendingClub](https://www.lendingclub.com/) dataset used in the previous two assignments.

In [51]:
loans = pd.read_csv('../Week3/lending-club-data.csv')

**2.** As before, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [52]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)

# just check if conversion done properly, before removing the 'bad_loans' col
print loans[['id', 'member_id', 'loan_amnt', 'bad_loans', 'safe_loans']].head(2)

# removing row traightforward
# to remove/drop a column, axis=1 denotes that we are referring to a column
# see http://chrisalbon.com/python/pandas_dropping_column_and_rows.html

loans = loans.drop('bad_loans', axis=1)

        id  member_id  loan_amnt  bad_loans  safe_loans
0  1077501    1296599       5000          0           1
1  1077430    1314167       2500          1          -1


**3.** We will be using the same 4 categorical features as in the previous assignment: 
1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

In the dataset, each of these features is a categorical feature. Since we are building a binary decision tree, we will have to convert this to binary data in a subsequent section using 1-hot encoding.

In [53]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

print features

['grade', 'term', 'home_ownership', 'emp_length']


In [54]:
print len(loans)
loans.head(5)

122607


,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## Subsample dataset to make sure classes are balanced

**4.** Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed = 1` so everyone gets the same results.

In [55]:
'''
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)
'''

'\nsafe_loans_raw = loans[loans[target] == 1]\nrisky_loans_raw = loans[loans[target] == -1]\n\n# Since there are less risky loans than safe loans, find the ratio of the sizes\n# and use that percentage to undersample the safe loans.\npercentage = len(risky_loans_raw)/float(len(safe_loans_raw))\nsafe_loans = safe_loans_raw.sample(percentage, seed = 1)\nrisky_loans = risky_loans_raw\nloans_data = risky_loans.append(safe_loans)\n\nprint "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))\nprint "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))\nprint "Total number of loans in our new dataset :", len(loans_data)\n'

**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in "[Learning from Imbalanced Data](http://www.ele.uri.edu/faculty/he/PDFfiles/ImbalancedLearning.pdf)" by Haibo He and Edwardo A. Garcia, *IEEE Transactions on Knowledge and Data Engineering* **21**(9) (June 26, 2009), p. 1263–1284. For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

82% of data samples were safe loans, and about 18% risky loans, so huge class imbalance. Given 82% of the loans are safe, if we predict that all loans are safe, we will be correct 82% of the time. That sounds like high accuracy, but is not. We need our accurcay to be better than 80% if we are to do do better than prior probability - background spread or distribution! Bit like in fog forecasting, fog only occurs 2% of days in a year, so if we forecast no fog every time, we will be correct 98% of the time. But 2% of the time we will be wrong - we have to determine if this false negatives is acceptable risk to industry.

## Transform categorical data into binary features

Since we are implementing **binary decision trees**, we transform our categorical data into binary data using 1-hot encoding, just as in the previous assignment. Here is the summary of that discussion:

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and GraphLab tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

**5.** This technique of turning categorical variables into binary variables is called one-hot encoding. Using the software of your choice, perform one-hot encoding on the four features described above. You should now have 25 binary features.

Since this code requires a few Python tricks, feel free to use this block of code as is - note it will only work with sframes!. Refer to the API documentation for a deeper understanding. Note that for some categorical features, more features are created than existing levels, for e.g home_ownership has 3 levels, 'OWN', 'RENT', 'MORTGAGE' but one-hot encoding creates 4 levels, an extra level called 'OTHER', so in effect creates 4 seperate features when we would have expected only 3 expected features. Another approach is using sklearns LabelEncoder().

Generally one-hot encoding appears to be better approach than using LabelEncoder which assigns different numerical values to seperate levels in a class. E.g home_ownership column values will be assighned numeric value 1 for 'RENT', 2 for 'OWN', 3 for 'MORTGAGE'. This often causes the decision tree classfier to treat the feature as numeric so we start to see splits like (home_ownership <= 2.5) which may not always have meaningful interpretation.

Thankfully One-hot encoding is supported in pandas as pd.get_dummies() !!
* Apply one-hot encoding to loans. see [this](https://gist.github.com/ramhiser/982ce339d5f8c9a769a0) and also [this](http://fastml.com/converting-categorical-data-into-numbers-with-pandas-and-scikit-learn/) 

In [56]:
# loans_data = risky_loans.append(safe_loans) #already appended earlier!

for feature in features:
    # Get one hot encoding of column feature
    one_hot = pd.get_dummies(loans[feature])
    
    # Drop column B as it is now encoded
    loans = loans.drop(feature, axis=1)
    
    # Join the encoded df
    loans = loans.join(one_hot)

The feature columns now look like this:

In [57]:
features = list(loans.columns)   #loans.columns return index object, force it to be list
# AttributeError: 'Index' object has no attribute 'remove' , so make features list as above
features.remove('safe_loans')  # Remove the response variable
features

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 ' 36 months',
 ' 60 months',
 'MORTGAGE',
 'OTHER',
 'OWN',
 'RENT',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year',
 'n/a']

In [58]:
print "Number of features (after binarizing categorical variables) = %s" % len(features)
print len(loans['A'])
loans['A'][0:10]

Number of features (after binarizing categorical variables) = 25
122607


0    0.0
1    0.0
2    0.0
3    0.0
4    1.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: A, dtype: float64

## Train-Validation split

**6.** We split the data into a train-validation split with 80% of the data in the training set and 20% of the data in the validation set. We use `seed=1` so that everyone gets the same result.

In [59]:
#train_data, validation_data = loans_data.random_split(.8, seed=1)

import json
with open('module-6-assignment-train-idx.json', mode='r') as file1: 
    train_idx = json.load(file1)   #reads entire file in one go into list

with open('module-6-assignment-validation-idx.json', mode='r') as file2: 
    validation_set_idx = json.load(file2)    #reads entire file in one go into list

file1.close()
file2.close()

# Now we need to grab all data cols from loans 
# with row indices train_idx in train set, similar for validation set
train_data           =  loans.iloc[train_idx] 
validation_set       =  loans.iloc[validation_set_idx]

print 'Training set         : %d data points' % len(train_data)
print 'Validation set       : %d data points' % len(validation_set)

Training set         : 37224 data points
Validation set       : 9284 data points


Note. Some elements in loans are included neither in train_data nor test_data. This is to perform sampling to achieve class balance.

Now proceed to the section "Decision tree implementation", skipping three (subsampling, one-hot encoding and train/test split - as already done above) sections below.

# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this.

## Early stopping condition 2: Minimum node size
In the tree building, we repeat the process of looking for the best predictor and best cutpoint/split point in order to split the data further so as to minimize the RSS or classification error within each of the resulting nodes. So every time, instead of splitting the entire predictor space, we split one of the two previously identified regions, left and right nodes, and repeat tree building process in these nodes. 

Again, we look to split one of these two intermediate nodes further, so as to minimize the eror rate/RSS. But when do we stop! The process continues until a stopping criterion is reached; for instance, we may continue until no region/node contains more than say five observations or sample data points. SO we define fn below that checks if present node meets this stopping criterion. If it does we stop tree building process on this branch/node and return this node as a leaf.

The function **reached_minimum_node_size** takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

**7.** This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the **decision_tree_create** function. Thus if a node being considered for further split has fewer than `min_node_size`, then it won't be split further, instead it wil be converted to a leaf.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below.

In [60]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
     return  ( len(data) <= min_node_size)

** Quiz question:** Given an intermediate node with 6 safe loans and 3 risky loans, if the `min_node_size` parameter is 10, what should the tree learning algorithm do next? 6+3=9. Given number of points in the node `len(data)=9` is less than the `min_node_size=10` , we wont consider this node for further split, instead convert it to a leaf.

In [61]:
#if  reached_minimum_node_size(data, min_node_size):         ## YOUR CODE HERE 
#        print "Early stopping condition 2 reached. Reached minimum node size."
#        return create_leaf(target_values)  ## YOUR CODE HERE

## Early stopping condition 3: Minimum gain in error reduction
The tree building process we used in last asignment may produce good predictions on the training
set, but is likely to overfit the data, leading to poor test set performance. This is because the resulting tree might be too complex. A smaller tree with fewer splits (that is, resulting in fewer nodes/leaves) might lead to lower variance and better interpretation at the cost of a little bias. One possible alternative is to build the tree only so long as the decrease in the error rate/RSS due to each split exceeds some (high) threshold. This strategy will result in smaller trees, but maybe is too short-sighted since a seemingly worthless split early on in the tree might be followed by a very good split—that is, a split that leads to a large reduction in error rate/RSS later on. see page 307 ISLR. Anyway we will implement function `error_reduction()` that will help us check if error reduction resulting from split under consideration will exceed some threshold. Ir reduction in error rate due to new split is lower than the set threshold, then we won't make the split and set itermediate node as a leaf instead.

The function **error_reduction** takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

**8.** This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below. 

In [62]:
def error_reduction(error_before_split, error_after_split):
    return (error_before_split-error_after_split)

** Quiz question:** Assume an intermediate node has 6 safe loans and 3 risky loans.  For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the **minimum gain in error reduction** parameter is set to 0.2, what should the tree learning algorithm do next? 

So here, we have 4 features, and we are trying to find out which feature we should split on. We want to find out if the decrease in the error rate/RSS due to each split being considered exceeds our (high) threshold set at 0.2.
We can see that, even with the best candidate for splitting feature, the error reduction is at most only 0.14; this decrease in RSS or error rate is less than the threshold `min_error_reduction=0.2`. So there is no point splitting the intermediate node. Set it as leaf and return. Note that if the decrease in error exceeded threshold 0.2, we would not convert 'this' node to a leaf, but we would proceed with the split.

In [63]:
#error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))

#if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:
#if  ( error_before_split - error_after_split ) <= min_error_reduction:   
#if  ( error_after_split - error_before_split ) > min_error_reduction:   
#        print "Early stopping condition 3 reached. Minimum error reduction."
#        return create_leaf(target_values)   ## YOUR CODE HERE#max error reduction = 0.14 = 0.33 - 0.9, 0.14 <= 0.2, 
#so not quite achieving as much error reduction as we want, so no point splitting further! 

## Grabbing binary decision tree helper functions from past assignment

**9.** Recall from the previous assignment that we wrote a function `intermediate_node_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

**Paste your code for `intermediate_node_num_mistakes` here**.

In [64]:
def intermediate_node_num_mistakes(labels_in_node):
    
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # print "Number of samples %d " %len(labels_in_node)
    # print "Samples class distribution %s " %(labels_in_node)
    # mistakes = None
    
    # Count the number of 1's (safe loans)
    # num_of_1s =      (loans['safe_loan'] == +1).sum()
    safe_loans_count = sum(labels_in_node == +1)
    # safe_loans_count = sum([1 if x == 1 else 0 for x in labels_in_node])
    
    # Count the number of -1's (risky loans)
    risky_loans_count = sum(labels_in_node == -1)
    # risky_loans_count = sum([1 if x == -1 else 0 for x in labels_in_node])
       
    
    ##uncomment if want to print some diagnostics...
    
    # print "(num risky: %d , num safe: %d) in the split." %(risky_loans_count,safe_loans_count)
    
    # Return the number of mistakes that the majority classifier makes.
    #maj_class = '+1' if safe_loans_count >= risky_loans_count else '-1'
    
    
    #mistakes = risky_loans_count if (maj_class == '+1') else safe_loans_count
    #mistakes = risky_loans_count if (safe_loans_count >= risky_loans_count) else safe_loans_count
    # what above line is doing is just trying to find minimum of the two count values!!!
    # hint
    # Since we are assuming majority class prediction, 
    # all the data points that are not in the majority class are considered mistakes
    # we don't even have to determine majority class
    # the bigger count will always be majority, lower count wud always be mistakes
    # mistakes = min(safe_loans_count, risky_loans_count)   #as easy as that!!
    # print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    
    # So for e.g if we have more safe loans than risky, majority vote wud be to classify the
    # whole lot as of safe class, and mistakes would be just count of the other (risky) class
    # so just return count of the smaller class sample, cause that wud always be minority class
    # and minority class always be misclassfied !!    
    
    '''
    # uncomment if want to print extra some diagnostics... 
    if (maj_class == '+1'):
        #mistakes = len(labels_in_node) - safe_loans_count
        mistakes = risky_loans_count
        print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    else:
        #mistakes = len(labels_in_node) - risky_loans_count
        mistakes = safe_loans_count
        print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    '''  
    return min(safe_loans_count, risky_loans_count)

**10.** We also wrote a function `best_splitting_feature` that finds the best feature to split on given the data and a list of features to consider. Note that here, we look at each feature available for split and find error rate if a split was made using that feature. End goal is finding a feature to split on that results in lowest/minimum classification error rate.

Then in the main decision tree building function we will check if the decrease in error rate caused by this split exceeds our target threshold of error_reduction we want to achive with the split. If it exceeds then we will make split.

**Paste your `best_splitting_feature` code here**.

In [65]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    # print "num_data_points = %d" %(num_data_points)
    # print data[features].head(1)
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        #print "\n************Trying out feature:", feature
        
        # The left split will have all data points where the feature value is 0
        left_split = data[ data[feature] == 0 ]
        #print "left split",left_split[feature] #this will just split out a list of 0's
        #print "num_data_points in (data[%s] == 0) i.e. left_split = %d " %(feature,len(left_split))
        
        # The right split will have all data points where the feature value is 1
        right_split =  data[ data[feature] == 1 ]
        #print "right split", right_split[feature] #this will just split out a list of 1's
        #print "num_data_points (data[%s] == 1) i.e. right_split = %d " %(feature,len(right_split))
        
        
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = intermediate_node_num_mistakes((left_split[target]))
        
        # What we are doing above is is trying to find out how many target class is +ve 
        # and how many target class is -ve for cases where the feature values are 0
        # so we try find majority class label where the feature values are all 0
        # Reuse function intermediate_node_num_mistakes()
        # this function determines how many samples data[ data[feature] == 0 ]
        # have +ve labels and how many -ve lables, then determines majority lable
        # say if majority were +ves, then mistakes is simply count of -ves in sample
        
             
        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes((right_split[target]))
        
        # Now repeat above step for all data points where data[ data[feature] == 1 ] 
        # i.e where the feature variable in question has all values 1
        # and find number of mistakes here same way as above (find majority class
        # and mistakes is just count of data points of the other minority class)
        
        #print num_data_points == (len(left_split) + len(right_split))                  
        
            
        # -------------------------------------------------------------------------    
        # Compute the classification error of this split.
        # i.e if we chose this feature to make a decision/split in tree 
        # then how do we measure how pure leaf nodes we will get
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes)/num_data_points
        #print "Error using %s for split is %0.4f " %(feature, error)
        
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            #print "\nFOUND LOWER ERROR using %s, error is %0.4f, previous best error %0.4f" \
            #                                 %(feature, error, best_error)
            best_feature = feature
            best_error = error
            
    print "\nBEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: %s, error rate: %0.4f" \
                                             %(best_feature, best_error)
    return best_feature # Return the best feature we found

**11.** Finally, recall the function `create_leaf` from the previous assignment, which creates a leaf node given a set of target values.  

**Paste your `create_leaf` code here**.

In [66]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None, # unknown at this stage
            'left' : None,              # unknown at this stage
            'right' : None,             # unknown at this stage
            'is_leaf': True     #set to True cause we are creating a leaf
           }  
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones =      len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =  +1        ## YOUR CODE HERE
    else:
        leaf['prediction'] =  -1        ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

## Incorporating new early stopping conditions in binary decision tree implementation

**12.** Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment.  In particular, you will write code to detect early stopping conditions 2 and 3.  You implemented above the functions needed to detect these conditions.  The 1st early stopping condition, **max_depth**, was implemented in the previous assigment and you will not need to reimplement this.  In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [67]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # STOPPING CONDITION 1: All nodes are of the same type.
    # (Check if there are NO or ZERO mistakes at current node.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. No mistakes - PURE node!"                
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)

     # STOPPING CONDITION 2: No more features to split on.
    # (check if there are remaining features to consider splitting on)
    # if len(remaining_features) is  0 then no features left in list    
    if remaining_features == []: #if remaining_features == 0:
        print "Stopping condition 2 reached. No remaining splitting features."   
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # EARLY STOPPING CONDITION 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    # EARLY STOPPING CONDITION 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size):         ## YOUR CODE HERE 
    # if ( len(data) <= min_node_size):
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)  ## YOUR CODE HERE
    
    # Find the best splitting feature 
    # we assume that this fn will always find n return a feature - so no error checking
    # returns feature that gives lowest classification error when split on that feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Now split on the best feature that we found. 
    left_split  = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # EARLY STOPPING CONDITION 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])   ## YOUR CODE HERE
    right_mistakes =intermediate_node_num_mistakes(right_split[target])    ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    print "Error B4 split: %0.4f, Error after split: %0.4f, B4-after=%0.4f" \
               %( error_before_split,  error_after_split, (error_before_split - error_after_split)) 
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:   ## YOUR CODE HERE
    #if  ( error_before_split - error_after_split ) <= min_error_reduction:   
    #if  ( error_after_split - error_before_split ) > min_error_reduction:   
        print "Early stopping condition 3 reached. Minimum error reduction."
        #print "Error B4 split: %0.4f, Error after split: %0.4f, B4-after=%0.4f" \
        #       %( error_before_split,  error_after_split, error_before_split - error_after_split) 
        return create_leaf(target_values)   ## YOUR CODE HERE 
    
    # when we are fully convinced that this split is worth it, 
    # remove feature so we don't try split on it again
    # and then build decision tree of the left and right branches
    remaining_features.remove(splitting_feature)  
    print "\n\nSPAWN NEW TREES (Left = %s, Right = %s) split on: %s" % (\
                      len(left_split), len(right_split),splitting_feature )
    
    
    # Create a leaf node if the split is "perfect" - not happen often!
    # Not checking for these - good - so we can create more pure nodes!!!
    '''
    if len(left_split) == len(data):
        print "CREATING A PURE LEFT LEAF NODE."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "CREATING A PURE RIGHT LEAF NODE"
        return create_leaf(right_split[target])    
    '''
    
    # Now enter recursive binary splitting -  a top-down, greedy approach
    # Start at top of the tree (point at which all observations belong to a single region) 
    # and then successively splits the predictor space; 
    # each split is indicated via two new branches further down on the tree    
    # remaining_features won't consider feature we split on above,
    # inc depth as we going one level deeper, target is just 'soft_loans'
    print "########### Growing left branch of tree further ##############"
    # Takes dataframe where value for predictor_feature (splitting_feature) is 0 and
    # subset of features and grows decision tree on that branch    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    
    print "########### Growing right branch of tree further ##############"
    # Takes dataframe where value for predictor_feature (splitting_feature) is 1 and
    # subset of features and grows decision tree on that branch        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        

    # now we have two dictionary objects left_tree and right_tree
    # so node we are at can't be a leaf, cause we have left and right branches - is_leaf FALSE
    # also store the left_tree and right_tree dictionaries at this node
    # all done in the return stmnt BLW
    # da magic of recursion happens in the return statement !!  
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a function to count the nodes in your tree:

In [68]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [69]:
small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Growing right branch of tree further ###########

## Build a tree!

**13.** Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

**Warning**: This code block may take a minute to learn. Call this model my_decision_tree_new.

In [70]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

**14.** Let's now train a tree model **ignoring early stopping conditions 2 and 3** so that we get the same tree as in the previous assignment.  To ignore these conditions, we set `min_node_size=0` and `min_error_reduction=-1` (a negative value).Call this model my_decision_tree_old. 

In [71]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

## Making predictions

**15.** Recall that in the previous assignment we implemented a function `classify` to classify a new point `x` using a given `tree`.

**Paste your `classify` code here**.

In [72]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

**16.** Now, let's consider the first example of the validation set and see what the `my_decision_tree_new` model predicts for this data point.

In [73]:
print validation_set.iloc[0]
print '\nPredicted class: %s ' % classify(my_decision_tree_new, validation_set.iloc[0])

safe_loans   -1.0
A             0.0
B             0.0
C             0.0
D             1.0
E             0.0
F             0.0
G             0.0
 36 months    0.0
 60 months    1.0
MORTGAGE      0.0
OTHER         0.0
OWN           0.0
RENT          1.0
1 year        0.0
10+ years     0.0
2 years       1.0
3 years       0.0
4 years       0.0
5 years       0.0
6 years       0.0
7 years       0.0
8 years       0.0
9 years       0.0
< 1 year      0.0
n/a           0.0
Name: 24, dtype: float64

Predicted class: -1 


**17.** Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [74]:
classify(my_decision_tree_new, validation_set.iloc[0], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
At leaf, predicting -1


-1

**18.** Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as `my_decision_tree_old`.

In [75]:
classify(my_decision_tree_old, validation_set.iloc[0], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
Split on B = 0.0
Split on C = 0.0
Split on D = 1.0
Split on E = 0.0
At leaf, predicting -1


-1

* my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 100, min_error_reduction=0.0)
* my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,
                                min_node_size = 0, min_error_reduction=-1)

** Quiz question:** For `my_decision_tree_new` trained with `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, is the prediction path for `validation_set[0]` shorter, longer, or the same as for `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?  

Much Shorter - only 2 splits.

**Quiz question:** For `my_decision_tree_new` trained with `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, is the prediction path for **any point** always shorter, always longer, always the same, shorter or the same, or longer or the same as for `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?

Should always be shorter. **any point** bit tricky, maybe `shorter or the same` is better!!

** Quiz question:** For a tree trained on **any** dataset using `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, what is the maximum number of splits encountered while making a single prediction?

assuming dataset large enough so min_node_size=100 not a huge constraint, and suppose  `min_error_reduction=0.0` is satisfied as well. we should be able to grow a tree upto max-depth 6, so at most max 6 splits!!!

## Evaluating the model

**19.** Now let us evaluate the model that we have trained. You implemented this evautation in the function `evaluate_classification_error` from the previous assignment.

Note that when using .apply() with pandas dataframe, [pandas.DataFrame.apply()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html), also need to supply axis info. See [also](http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html)

* axis : {0 or ‘index’, 1 or ‘columns’}, default 0
 *        0 or ‘index’: apply function to each column
 *        1 or ‘columns’: apply function to each row

In [76]:
def evaluate_classification_error(tree, data,target='safe_loans'):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    # correct = sum(prediction == data[target])
    # error = len(data) - correct
    # return 1.0*error/len(data)
    
    return 1.0 * sum(prediction != data[target])/len(data)

**20.** Now, let's use this function to evaluate the classification error of `my_decision_tree_new` on the **validation_set**.

In [77]:
round( evaluate_classification_error(my_decision_tree_new, validation_set), 5)

0.38367

**21.** Now, evaluate the validation error using `my_decision_tree_old`.

In [78]:
round(evaluate_classification_error(my_decision_tree_old, validation_set),5)

0.38378

**Quiz question:** Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?  Only a fraction lower!!! We expect it to be lower, cause old tree was overfitting, so whiole its train error wud always be lower, its valid/test error should be higher as it won't be able to generalise well. New tree avoids overfitting when learning the tree, so should be able to generalise better and probbaly achieve lower validation/test errors on unseen data.

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

**2.** Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

** Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [79]:
model_1 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 2, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Growing right branch of tree further ###########

In [80]:
model_2 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                 max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

In [81]:
model_3 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                 max_depth = 14, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

### Evaluating the models

**23.** Let us evaluate the models on the **train** and **validation** data. Let us start by evaluating the classification error on the training data:

In [82]:
print "Training data, classification error (model 1):", \
     evaluate_classification_error(model_1, train_data)
print "Training data, classification error (model 2):", \
     evaluate_classification_error(model_2, train_data)
print "Training data, classification error (model 3):", \
     evaluate_classification_error(model_3, train_data)

Training data, classification error (model 1): 0.400037610144
Training data, classification error (model 2): 0.381850419084
Training data, classification error (model 3): 0.374462712229


**24.** Now evaluate the classification error on the validation data.

In [83]:
print "Validation data, classification error (model 1):", \
evaluate_classification_error(model_1, validation_set)
print "Validation data, classification error (model 2):", \
evaluate_classification_error(model_2, validation_set)
print "Validation data, classification error (model 3):", \
evaluate_classification_error(model_3, validation_set)

Validation data, classification error (model 1): 0.398104265403
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.380008616975


**Quiz Question:** Which tree has the smallest error on the validation data?
Large and complex tree with max_depth=14, model_3

**Quiz Question:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?
Yes, the Large and complex tree with max_depth=14, model_3

**Quiz Question:** Is it always true that the tree with the lowest classification error on the **training** set will result in the lowest classification error in the **validation** set? 
No, in fact we expect the opposite usually. Model or tree with lowest classification error on the **training** set may be overfitting and could result in the higher classification error in the **validation** set.


### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

**25.** A function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [84]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

**26.** Using the function count_leaves, compute the number of nodes in `model_1`, `model_2`, and `model_3`.

In [85]:
print count_leaves(model_1)
print count_leaves(model_2)
print count_leaves(model_3)

4
41
341


**Quiz question:** Which tree has the largest complexity? model_3, the tree with largest num of nodes.
    

**Quiz question:** Is it always true that the most complex tree will result in the lowest classification error in the **validation_set**?   Not always true. Depends on dataset. Generally speaking, the most complex tree will ALWAYS have lowest classification error in the **training_set**, but same complex tree can have the worst/highest classification error in the **validation_set**.

# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

**27.** Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

** Note:** Each tree can take up to 30 seconds to train.

In [86]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

In [87]:
model_5 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

In [88]:
model_6 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=5.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747
Early stopping condition 3 reached. Minimum error reduction.


**28.** Calculate the accuracy of each model (**model_4**, **model_5**, or **model_6**) on the validation set. 

In [89]:
print "Validation data, classification error (model 4):", \
evaluate_classification_error(model_4, validation_set)
print "Validation data, classification error (model 5):", \
evaluate_classification_error(model_5, validation_set)
print "Validation data, classification error (model 6):", \
evaluate_classification_error(model_6, validation_set)

Validation data, classification error (model 4): 0.383778543731
Validation data, classification error (model 5): 0.383778543731
Validation data, classification error (model 6): 0.503446790177


**29.** Using the **`count_leaves`** function, compute the number of leaves in each of each models in (**model_4**, **model_5**, and **model_6**). 

In [90]:
print count_leaves(model_4)
print count_leaves(model_5)
print count_leaves(model_6)

41
13
1


**Quiz Question:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity? 
model_4 is most complex - as it has the most num of leaves. Thus many complex tree traversals to form predictions.

Did this match your expectation?  Yes, it had no stopping condition for min error reduction, so all splits were considered, maybe even those that led to increase in cllasification error rate.

**Quiz Question:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity? Should you pick **model_5** over **model_4**? No because error rate over 50% for two class prediction is worse than random guessing!


# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

**30.** Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

** Note:** Each tree can take up to 30 seconds to train.

In [91]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

In [92]:
model_8 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 2000, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE:  36 months, error rate: 0.4216
Error B4 split: 0.4963, Error after split: 0.4216, B4-after=0.0747


SPAWN NEW TREES (Left = 9223, Right = 28001) split on:  36 months
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: A, error rate: 0.3467
Error B4 split: 0.3492, Error after split: 0.3467, B4-after=0.0025


SPAWN NEW TREES (Left = 9122, Right = 101) split on: A
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

BEST FEATURE TO SPLIT ON THAT PRODUCES LOWEST ERROR RATE: B, error rate: 0.3463
Error B4 split: 0.3463, Error after sp

In [93]:
model_9 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 50000, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 2 reached. Reached minimum node size.


**31.** Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_set**.

In [94]:
print "Validation data, classification error (model 7):", \
evaluate_classification_error(model_7, validation_set)
print "Validation data, classification error (model 8):", \
evaluate_classification_error(model_8, validation_set)
print "Validation data, classification error (model 9):", \
evaluate_classification_error(model_9, validation_set)

Validation data, classification error (model 7): 0.383778543731
Validation data, classification error (model 8): 0.384532529082
Validation data, classification error (model 9): 0.503446790177


**32.** Using the `count_leaves` function, compute the number of leaves in each of each models (**model_7**, **model_8**, and **model_9**). 

In [95]:
print count_leaves(model_7)
print count_leaves(model_8)
print count_leaves(model_9)

41
19
1


**Quiz Question:** Using the results obtained in this section, which model (**model_7**, **model_8**, or **model_9**) would you choose to use? model_8 - just right!!! model_7 is too complex - many nodes, and given min_node_size=0, in theory this can lead to splits resulting in a sample/data point in each node in the worst case - so worst case of overfitting. On the other hand, model_8 has all data in one node - useless for prediction.